# RDF to MARC


In [43]:
import asyncio
import csv
import io
import js2py
import pandas as pd
import pymarc
import rdflib
import requests
import zipfile
from typing import Any, Optional

In [4]:
stanford_export = requests.get("https://sinopia-exports-production.s3-us-west-2.amazonaws.com/stanford_2020-04-26T00:02:02.114Z.zip",
                              stream=True)
stanford_zip = zipfile.ZipFile(io.BytesIO(stanford_export.content))


In [5]:
school_work = requests.get('https://trellis.sinopia.io/repository/stanford/da12a617-7832-4c08-bb3d-4f3b823e3eb0')
stanford_graph = rdflib.ConjunctiveGraph()
stanford_graph.parse(data=school_work.text, format='turtle')
        
    

<Graph identifier=N0368ecd7312b47dd8fc982df3d8be528 (<class 'rdflib.graph.Graph'>)>

In [6]:
leader_mappings[0]

{'Work (W), Instance (I), Item (Item)': '',
 'Path to value': '',
 'MARC destination': '00-04',
 'Conversion': '"     "',
 'Notes': 'All blanks for MARCXML'}

In [9]:
leader = pd.read_csv('../src/bf2marcSpec-LDR-001-005-v1.0.csv')

In [10]:
leader

,"Work (W), Instance (I), Item (Item)",Path to value,MARC destination,Conversion,Notes
0,NaN,NaN,00-04,""" """,All blanks for MARCXML
1,W,adminMetadata/AdminMetadata/status/Status/code,05,"(value == """") ? ""n"" : value","Default ""n"""
2,W,rdf:type/@rdf:resource,06,"if ((value == ""bf:NotatedMusic"") && (Instance/...","Default ""a"""
3,I,issuance/Issuance/@rdf:about,07,"if (Instance/rdf:type/@rdf:resource == ""bf:Col...","Default ""m"". Comes from either the issuance or..."
4,I,rdf:type/@rdf:resource,08,"(value == ""bf:Archival"") ? ""a"" : "" ""","Default "" """
5,NaN,NaN,09-16,"""a22 """,Defaults for MARCXML
6,W,adminMetadata/AdminMetadata/bflc:encodingLevel...,17,"if (@rdf:about == ""http://id.loc.gov/vocabular...","Default ""u"". Conversion may come from either t..."
7,W,adminMetadata/AdminMetadata/descriptionConvent...,18,"if (@rdf:about == ""http://id.loc.gov/vocabular...","Default ""u"". Conversion may come from either t..."
8,NaN,NaN,19,""" """,nac
9,NaN,NaN,20-23,"""4500""",Default for MARCXML


In [11]:
leader[:5]

,"Work (W), Instance (I), Item (Item)",Path to value,MARC destination,Conversion,Notes
0,NaN,NaN,00-04,""" """,All blanks for MARCXML
1,W,adminMetadata/AdminMetadata/status/Status/code,05,"(value == """") ? ""n"" : value","Default ""n"""
2,W,rdf:type/@rdf:resource,06,"if ((value == ""bf:NotatedMusic"") && (Instance/...","Default ""a"""
3,I,issuance/Issuance/@rdf:about,07,"if (Instance/rdf:type/@rdf:resource == ""bf:Col...","Default ""m"". Comes from either the issuance or..."
4,I,rdf:type/@rdf:resource,08,"(value == ""bf:Archival"") ? ""a"" : "" ""","Default "" """


In [19]:
js2py.eval_js(leader["Conversion"][0])

'     '

In [23]:
js2py.eval_js('("" == "") ? "n" : "for"')

'n'

In [24]:
leader["Conversion"].head()

0                                              "     "
1                          (value == "") ? "n" : value
2    if ((value == "bf:NotatedMusic") && (Instance/...
3    if (Instance/rdf:type/@rdf:resource == "bf:Col...
4                 (value == "bf:Archival") ? "a" : " "
Name: Conversion, dtype: object

In [28]:
def conversion(js_template, value:str) -> Any:
    js_str = js_template.replace("value", f"\"{value}\"")
    return js2py.eval_js(js_str)

In [35]:
def rdf2marc(graph: rdflib.Graph) -> pymarc.Record:
    leader_str = leader(graph, 'bf2marcSpec-LDR-001-005-v1.0.csv')
    record = pymarc.Record(leader=leader_str, file_encoding='utf-8')
    return record

In [32]:
?pymarc.Record

Init signature:
pymarc.Record(
    data='',
    to_unicode=True,
    force_utf8=False,
    hide_utf8_warnings=False,
    utf8_handling='strict',
    leader='                        ',
    file_encoding='iso8859-1',
)
Docstring:     
A class for representing a MARC record.

Each Record object is made up of multiple Field objects. You'll probably want to look
at the docs for :class:`Field <pymarc.record.Field>` to see how to fully use a Record
object.

Basic usage:

.. code-block:: python

    field = Field(
        tag = '245',
        indicators = ['0','1'],
        subfields = [
            'a', 'The pragmatic programmer : ',
            'b', 'from journeyman to master /',
            'c', 'Andrew Hunt, David Thomas.',
        ])

    record.add_field(field)

Or creating a record from a chunk of MARC in transmission format:

.. code-block:: python

    record = Record(data=chunk)

Or getting a record as serialized MARC21.

.. code-block:: python

    raw = record.as_marc()

You'll nor

In [31]:
print(rdf2marc())

=LDR            22        4500



In [ ]:
def leader(graph: rdflib.Graph, map_csv: str) -> str:
    leader_df = pd.read_csv(map_csv)
    js2py.eval_js(leader_df["Conversion"][0])

In [38]:
f1XX_df = pd.read_csv("../src/bf2marcSpec-1XX-6XX-7XX-8XX-v1.0.csv")

In [39]:
f1XX_df.head()

,"Work (W), Instance (I), Item (Item)",Path to value,MARC destination,MARC Repeatability,Conversion,Notes
0,W,contribution/bflc:PrimaryContribution/agent/(m...,100,NR,NaN,Each contribution/bflc:Contribution/agent/mads...
1,W,contribution/bflc:PrimaryContribution/agent/ma...,100 I1,NR,"""1""",NaN
2,W,contribution/bflc:PrimaryContribution/agent/ma...,100 I1,NR,"""3""",NaN
3,W,contribution/bflc:PrimaryContribution/agent/(m...,100 I2,NR,""" """,NaN
4,W,contribution/bflc:PrimaryContribution/agent/(m...,100 $a,NR,NaN,NaN


In [41]:
f1XX_df.keys()

Index(['Work (W), Instance (I), Item (Item)', 'Path to value',
       'MARC destination', 'MARC Repeatability', 'Conversion', 'Notes'],
      dtype='object')

In [47]:
def conversion(js_template, value: Optional[str] = None) -> Any:
    js_str = js_template.replace("value", f"\"{value}\"")
    print(f"js {js_str}")
    return js2py.eval_js(js_str)

In [49]:
conversion('"  "')

js "  "


'  '